https://wybory.gov.pl/pe2019/pl/wyniki/protokol/485230

https://selenium-python.readthedocs.io/api.html

https://stackoverflow.com/questions/23110383/how-to-dynamically-build-a-json-object-with-python

Install chromedriver

In [114]:
import pandas as pd
from selenium import webdriver
import os
import json

projectDirectory = "/Users/nataliakusek/PycharmProjects/european-elections-2019-poland"

In [160]:
options = webdriver.ChromeOptions() 
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [74]:
obwody_glosowania = pd.read_csv(os.path.join(projectDirectory, 'dane_statyczne', 'obwody_glosowania.csv'), sep = ';')

In [92]:
TERYT_codes = obwody_glosowania['TERYT gminy'].unique()
print(TERYT_codes)

[ 20101  20102  20103 ... 326101 326201 326301]


In [93]:
def get_protocols_ids(gmina_id):
    driver.get('https://wybory.gov.pl/pe2019/pl/wyniki/gm/' + gmina_id)
    results = driver.find_elements_by_css_selector("div.proto tbody tr")
    return list(map(lambda elem: elem.get_attribute("data-id"), results))
    
print(get_protocols_ids('20101'))

['457895', '457896', '457897', '457898', '457899', '457900', '457901', '457902', '457903', '457904', '457905', '457906', '457907', '457908', '457909', '457910', '457911', '457912', '457913', '457914', '457915', '485228', '485229', '485230']


In [100]:
def extract_card(card_web_element):
    tds = card_web_element.find_elements_by_css_selector('td')
    card = {}
    card['id'] = tds[0].text
    card['name'] = tds[1].text
    card['count'] = tds[2].text
    return card

def extract_info(info_web_element):
    dds = info_web_element.find_elements_by_css_selector('dd')
    info = {}
    info['Siedziba'] = dds[0].text
    info['Numer'] = dds[1].text
    info['Okręg wyborczy'] = dds[2].text
    info['Kod TERYT'] = dds[3].text
    info['Województwo'] = dds[4].text
    info['Powiat'] = dds[5].text
    info['Gmina'] = dds[6].text
    return info

def extract_candidate(candidate_web_element):
    tds = candidate_web_element.find_elements_by_css_selector('td')
    candidate = {}
    candidate['name'] = tds[1].text
    candidate['votes'] = tds[2].text
    return candidate

def extract_list(list_web_element):
    trs = list_web_element.find_elements_by_css_selector('tbody tr')
    foot = list_web_element.find_elements_by_css_selector('tfoot tr td')
    h3 = list_web_element.find_elements_by_css_selector('h3')
    lists = {}
    lists['name'] = h3[0].text
    lists['candidates'] = list(map(lambda candidate: extract_candidate(candidate), trs))
    lists['total votes'] = foot[2].text
    return lists

def get_protocol_data(protocol_id):
    driver.get('https://wybory.gov.pl/pe2019/pl/wyniki/protokol/' + protocol_id)
    info = driver.find_element_by_xpath('//div[contains(@class, "pro")]/dl[contains(@class, "info")]')
    info = extract_info(info)
    cards = driver.find_elements_by_css_selector("div.cards table tbody tr")
    cards = list(map(lambda card: extract_card(card), cards))
    lists = driver.find_elements_by_css_selector("div.lists > div")
    lists = list(map(lambda list: extract_list(list), lists))
    
    protocol = {}
    protocol['id'] = protocol_id
    protocol['info'] = info
    protocol['cards'] = cards
    protocol['lists'] = lists
    return protocol

print(get_protocol_data('457895'))

{'id': '457895', 'info': {'Siedziba': 'Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-700 Bolesławiec', 'Numer': '1', 'Okręg wyborczy': 'Nr 12 [Wrocław]', 'Kod TERYT': '020101', 'Województwo': 'dolnośląskie', 'Powiat': 'bolesławiecki', 'Gmina': 'm. Bolesławiec'}, 'cards': [{'id': '1.', 'name': 'Komisja otrzymała kart do głosowania', 'count': '1213'}, {'id': '2.', 'name': 'Liczba wyborców uprawnionych do głosowania (umieszczonych w spisie, z uwzględnieniem dodatkowych formularzy) w chwili zakończenia głosowania', 'count': '1543'}, {'id': '2a.', 'name': 'w tym umieszczonych w części A spisu wyborców (obywatele polscy)', 'count': '1543'}, {'id': '2b.', 'name': 'w tym umieszczonych w części B spisu wyborców (obywatele UE niebędący obywatelami polskimi)', 'count': '0'}, {'id': '3.', 'name': 'Nie wykorzystano kart do głosowania', 'count': '577'}, {'id': '4.', 'name': 'Liczba wyborców, którym wydano karty do głosowania (liczba podpisów w spisie oraz adnotacje „odmowa podpisu”)', 'count': '636'}

In [163]:
for gmina_id in TERYT_codes[:2]:
    if not os.path.isfile(os.path.join(projectDirectory, 'wyniki', '{}.json'.format(gmina_id))):
        gmina_protocols = {}
        protocol_ids = get_protocols_ids(str(gmina_id))
        for protocol_id in protocol_ids:
            protocol_data = get_protocol_data(protocol_id)
            gmina_protocols[protocol_id] = protocol_data
        with open(os.path.join(projectDirectory, 'wyniki', '{}.json'.format(gmina_id)), 'w', encoding='utf8') as gmina_protocol:
            json.dump(gmina_protocols, gmina_protocol, ensure_ascii=False)


In [137]:
path = os.getcwd()

In [ ]:
{
    "id": 457895,
    "info": {
        "Siedziba": "Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-700 Bolesławiec",
        ...
    },
    "cards": [
        {'id': '1.', 'name': 'Komisja otrzymała kart do głosowania', 'count': '1213'}, 
        {'id': '2.', 'name': 'Liczba wyborców uprawnionych do głosowania (umieszczonych w spisie, z uwzględnieniem dodatkowych formularzy) w chwili zakończenia głosowania', 'count': '1543'}, 
        {'id': '2a.', 'name': 'w tym umieszczonych w części A spisu wyborców (obywatele polscy)', 'count': '1543'}, {'id': '2b.', 'name': 'w tym umieszczonych w części B spisu wyborców (obywatele UE niebędący obywatelami polskimi)', 'count': '0'}, {'id': '3.', 'name': 'Nie wykorzystano kart do głosowania', 'count': '577'}, {'id': '4.', 'name': 'Liczba wyborców, którym wydano karty do głosowania (liczba podpisów w spisie oraz adnotacje „odmowa podpisu”)', 'count': '636'}, {'id': '4a.', 'name': 'w tym w części A spisu wyborców (obywatele polscy)', 'count': '636'}, {'id': '4b.', 'name': 'w tym w części B spisu wyborców (obywatele UE niebędący obywatelami polskimi)', 'count': '0'}, {'id': '5.', 'name': 'Liczba wyborców głosujących przez pełnomocnika (liczba kart do głosowania wydanych na podstawie otrzymanych przez komisję aktów pełnomocnictwa)', 'count': '0'}, {'id': '6.', 'name': 'Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania', 'count': '1'}, {'id': '7.', 'name': 'Liczba wyborców, którym wysłano pakiety wyborcze', 'count': '0'}, {'id': '8.', 'name': 'Liczba otrzymanych kopert zwrotnych', 'count': '0'}, {'id': '8a.', 'name': 'Liczba kopert zwrotnych, w których nie było oświadczenia o osobistym i tajnym oddaniu głosu', 'count': '0'}, {'id': '8b.', 'name': 'Liczba kopert zwrotnych, w których oświadczenie nie było podpisane przez wyborcę', 'count': '0'}, {'id': '8c.', 'name': 'Liczba kopert zwrotnych, w których nie było koperty na kartę do głosowania', 'count': '0'}, {'id': '8d.', 'name': 'Liczba kopert zwrotnych, w których znajdowała się niezaklejona koperta na kartę do głosowania', 'count': '0'}, {'id': '8e.', 'name': 'Liczba kopert na kartę do głosowania wrzuconych do urny', 'count': '0'}, {'id': '9.', 'name': 'Liczba kart wyjętych z urny', 'count': '636'}, {'id': '9a.', 'name': 'w tym liczba kart wyjętych z kopert na karty do głosowania', 'count': '0'}, {'id': '10.', 'name': 'Liczba kart nieważnych (innych niż urzędowo ustalone lub nieopatrzonych pieczęcią obwodowej komisji wyborczej)', 'count': '0'}, {'id': '11.', 'name': 'Liczba kart ważnych', 'count': '636'}, {'id': '12.', 'name': 'Liczba głosów nieważnych (z kart ważnych)', 'count': '1'}, {'id': '12a.', 'name': 'w tym z powodu postawienia znaku „X” obok nazwiska dwóch lub większej liczby kandydatów z różnych list', 'count': '1'}, {'id': '12b.', 'name': 'w tym z powodu niepostawienia znaku „X” obok nazwiska żadnego kandydata', 'count': '0'}, {'id': '12c.', 'name': 'w tym z powodu postawienia znaku „X” wyłącznie obok nazwiska kandydata na liście, której rejestracja została unieważniona', 'count': '0'}, {'id': '13.', 'name': 'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów (z kart ważnych)', 'count': '635'}
    ],
    "lists": [
        {
            'name': "KOMITET WYBORCZY WYBORCÓW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY" ,
            'candidates': [
                { 'name': "IWASZKIEWICZ Robert Jarosław", 'votes': 12 }
            ]
            'totalVotes': 35
        },
        ...
    ]
}